# Non-Negative Least Squares

## Introduction

In a non-negative least squares (NNLS) problem, we are given data $F \in \mathbf{R}^{p \times q}$ and $g \in \mathbf{R}^p$ and seek a vector $z \in \mathbf{R}_+^q$ such that $Fz$ is close to $g$. Closeness is defined as the sum of the squared differences, also known as the $\ell_2$-norm squared $\|Fz - g\|_2^2$.

We find the optimal $z$ by solving the problem

$$\begin{array}{ll} \text{minimize} & \|Fz - g\|_2^2 \\ \text{subject to} & z \geq 0. \end{array}$$

## Reformulate and Solve Problem

The problem can be written in standard form with

$$f_1(x_1) = \|Fx_1 - g\|_2^2, \quad f_2(x_2) = I_+(x_2),$$
$$A_1 = I, \quad A_2 = -I, \quad b = 0,$$

where $x_1,x_2 \in \mathbf{R}^p$. Here $I_+: \mathbf{R}^q \rightarrow \mathbf{R} \cup \{\infty\}$ is the set indicator of the non-negative reals, i.e.,

$$I_+(x_2) = \begin{cases} 0 & x_2 \geq 0 \\ \infty & \text{otherwise}. \end{cases}$$

We solve an instance with $p = 150$ and $q = 300$. The matrix $F$ is sparse with 0.1% non-zero entries, which are drawn IID from $N(0,1)$, and the vector $g$ is drawn from $N(0,I)$.

In [1]:
import numpy as np
from scipy import sparse
from a2dr import a2dr
from a2dr.proximal import *

np.random.seed(1)

# Problem data.
p, q = 150, 300 
density = 0.001
F = sparse.random(p, q, density = density, data_rvs = np.random.randn)
g = np.random.randn(p)

# Convert problem to standard form.
prox_list = [lambda v, t: prox_sum_squares_affine(v, t, F, g), prox_nonneg_constr]
A_list = [sparse.eye(q), -sparse.eye(q)]
b = np.zeros(q)

# Solve with A2DR.
a2dr_result = a2dr(prox_list, A_list, b)

# Print solution.
z_star = a2dr_result["x_vals"][-1]
obj = np.linalg.norm(F.dot(z_star) - g, 2)**2
print("\nObjective value:", obj)
print("Fraction of nonzeros in z:", np.count_nonzero(z_star)/q)
print("Optimal z:", z_star)

----------------------------------------------------------------
a2dr v0.1 - Prox-Affine Distributed Convex Optimization Solver
                 (c) Anqi Fu, Junzi Zhang
                 Stanford University, 2019
----------------------------------------------------------------
### Preconditioning starts ... ###
### Preconditioning finished.  ###
max_iter = 3000, t_init (after preconditioning) = 1.73, eps_abs = 1e-06, eps_rel = 1e-08
precond = True, ada_reg = True, anderson = True, m_accel = 10, lam_accel = 1e-08
aa_method = lstsq, D_safe = 1000000.00, eps_safe = 1e-06, M_safe = 30
variables n = 600, constraints m = 300
nnz(A) = 600
Setup time: 2.33e-02
----------------------------------------------------
 iter | total res | primal res | dual res | time (s)
----------------------------------------------------
     0| 1.61e+00    2.25e-01     1.59e+00   5.56e-02
   100| 9.14e-03    8.75e-05     9.14e-03   7.23e-01
   200| 4.41e-04    4.48e-06     4.41e-04   1.37e+00
   300| 3.53e-05    2